# Lie Group SO(3) Applied to Quadcopter
This notebook will go through the dynamics of a quadrotor when using the SO(3) Lie Group attitude representation as well as how an MPC controller can be designed for the system. The intention of this notebook is to help solidfy the ideas we have been learning on Lie Groups and to show an example of how they can be used in practice.

### Table of Contents

1. [Nomenclature](#Nomenclature)
1. [Quadcopter Dynamics](#Quadcopter-Dynamics)
 1. [Kinematics on the Manifold](#Kinematics-on-the-Manifold)
1. [Control Design](#Control-Design)
 1. [Derivation of Error State Dynamics](#Derivation-of-Error-State-Dynamics)
 1. [Summary of Error State Dynamics](#Summary-of-Error-State-Dynamics)
 1. [Jacobian Linearization](#Jacobian-Linearization)
 1. [MPC Design](#MPC-Design)
1. [Simulation](#Simulation)
 1. [Framework](#Framework)
 1. [Results](#Results)
    1. [Hourglass Waypoint Path Videos](#Hourglass-Waypoint-Path-Videos)
    1. [Hourglass Waypoint Path Images](#Hourglass-Waypoint-Path-Images)
    1. [Angle Wrapping Videos](#Angle-Wrapping-Videos)
    1. [Angle Wrapping Images](#Angle-Wrapping-Images)
    1. [Smart Angle Videos](#Smart-Angle-Videos)
    1. [Smart Angle Images](#Smart-Angle-Images)
$\newcommand{\b}[1]{\textbf{#1}}$
$\newcommand{\bs}[1]{\boldsymbol{#1}}$

## Nomenclature
This section defines the coordinate frames, variables, and operators that will be used throughout this notebook. All variable descriptors (subscripts and superscripts) are established here for the entire notebook. If there are no descriptors on a variable, they should be assumed to be the same as the descriptors on the respective variables in this section.

inertial frame: global origin following the NED convention

vehicle frame: located at the quadcopter's center of gravity following the NED convention

body frame: located at the quadcopter's COG with x-axis pointing to front propeller, y-axis pointing to right propeller, and z-axing pointing to the underside

$\newcommand{\b}[1]{\textbf{#1}}$
$\b{p}_{b/i}^i$: position of quadrotor's body frame with respect to the inertial frame, expressed in the inertial frame

$\b{v}_{b/i}^b$: velocity of body frame with respect to inertial frame, expressed in the body frame

$R_b^v$: rotation matrix describing the attitude of the body frame, expressed in the vehicle frame

$\bs{\phi}_{b}^v$: vectorized rotation describing attidude of the body frame, expressed in the vehicle frame

$\bs{\omega}_{b/i}^b$: angular velocity of the body frame with respect to the inertial frame, expressed in the body frame

$f^b$: total force produced from all 4 motors expressed in body frame, not a vector (always acts along $z^b$-axis)

$\bs{\tau}^b$: torque about 3 axes of body frame expressed in body frame

$\hat{\b{i}},\hat{\b{j}},\hat{\b{k}}$: orthonormal unit vectors representing the x, y, and z axes, respectively (e.g. $\hat{\b{k}} = [0,0,1]^T$)

$[\cdot]_\times$: skew-symmetric operator, used as a cross product ($[\b{x}]_x \b{y} = \b{x} \times \b{y}$)

$[\cdot]^\wedge$: hat operator - maps from $\mathbb{R}^3$ to Lie Algebra $[\mathbb{R}^3\rightarrow{\mathfrak so}(3)]$

$[\cdot]^\vee$: vee operator - maps from Lie Algebra to $\mathbb{R}^3 [{\mathfrak so}(3)\rightarrow \mathbb{R}^3]$

$\log()$: matrix logarithmic operator - maps from Lie Group to Lie Algebra [SO(3) $\rightarrow{\mathfrak so}(3)$]

$\exp()$: matrix exponential operator - maps from Lie Algebra to Lie Group [${\mathfrak so}(3)\rightarrow$ SO(3)]

![Lie Operators Aid](files/Lie_operators_aid.png)

[TOC](#Table-of-Contents)

## Quadcopter Dynamics
For more detail on how these equations are derived, see [Dr. Beard's PDF](https://scholarsarchive.byu.edu/cgi/viewcontent.cgi?article=2324&context=facpub).

\begin{eqnarray}
\dot{\b{p}}_{b/i}^i &=& R_b^v \b{v}_{b/i}^b \\
\dot{\b{v}}_{b/i}^b &=& [\b{v}_{b/i}^b]_\times \bs{\omega}_{b/i}^b -\frac{f^b}{m}\hat{\b{k}} + R_v^b \b{g}^v - \frac{\mu}{m} \b{v}_{b/i}^b \\
\dot{R}_b^v &=& R_b^v [\bs{\omega}_{b/i}^b]^\wedge\\
\dot{\bs{\omega}}_{b/i}^b &=& J^{-1} ([-\bs{\omega}_{b/i}^b]_\times J \bs{\omega}_{b/i}^b + \bs{\tau}^b) \\
\end{eqnarray}

[TOC](#Table-of-Contents)

### Kinematics on the Manifold
This derivation is very similar to that from [Unit 1 - Chapter 5: Optimal Control](../Unit-1-SO2/1.5-Optimal-Control.ipynb#Kinematics-Revisited-from-a-Manifold-Perspective). It is taking the time derivative of the already vectorized rotation, expressing the body frame (which changes with time) in the vehicle frame (which has a fixed rotation with respect to time). The final equation below replaces the kinematic equation on the SO(3) manifold, $R=R[\bs{\omega}]^\wedge$, with a vectorized kinematic equation that can then be used in the MPC controller.

$\newcommand{lp}{\left(}$
$\newcommand{rp}{\right)}$
\begin{eqnarray}
\frac{d}{dt}\bs{\phi}_{b(t)}^v &=& \lim_{\Delta t \to 0} \frac{1}{\Delta t} \lp [\log\lp [R_{b(t + \Delta t)}^v]^{-1} \circ R_{b(t)}^v \rp]^\vee \rp \\
&=& \lim_{\Delta t \to 0} \frac{1}{\Delta t} \lp [\log\lp R_v^{b(t + \Delta t)} \circ R_{b(t)}^v \rp]^\vee \rp \\
&=& \lim_{\Delta t \to 0} \frac{1}{\Delta t} \lp 
[\log\lp\exp\lp[\bs{\delta}_{b(t)}^{b(t + \Delta t)}]^\wedge\rp \circ R_v^{b(t)} \circ R_{b(t)}^v \rp]^\vee 
\rp \\
&=& \lim_{\Delta t \to 0} \frac{1}{\Delta t} \lp [\log\lp\exp\lp[\bs{\delta}_{b(t)}^{b(t + \Delta t)}]^\wedge\rp \rp]^\vee \rp \\
&=& \lim_{\Delta t \to 0} \frac{1}{\Delta t} \lp \Delta t \bs{\omega}_{b(t)/i}^{b(t)} \rp \\
&=& \bs{\omega}_{b(t)/i}^{b(t)}
\end{eqnarray}

[TOC](#Table-of-Contents)

## Control Design
In order to use MPC, error-state dynamics need to be used. This is because MPC uses convex optimization to minimize a cost function. Generally, MPC will minimize states and inputs. In order to be unbiased, the terms being minimized should be driven to 0. Using error-states allows the optimization solver to drive the error to 0.

The derivation below shows how we will be using $R_r$, which is the reference rotation matrix expressed in the vehicle frame. Note that the descriptors on each variable indicating frame of reference are dropped from here on out for readability, but the variables are all used as described in the [Nomenclature](#Nomenclature) section. 

Box-minus explained: tilde-states are defined here as actual-state minus reference-state. Rotation matrices can't be added or subtracted together which means you can't say $\tilde{R}=R-R_r$. This is why a combination of the log and vee operators are used to map from SO(3) to $\mathbb{R}^3$. This would suggest that the equation would be $\tilde{\bs{\phi}} = [\log(R)]^\vee - [\log(R_r)]^\vee$; however, $[\log(R)]^\vee - [\log(R_r)]^\vee$ is an invalid operation and $[\log(R_r^T R)]^\vee$ needs to be used instead. The way I make sense of this is remembering that $\log(x)-\log(y) = \log(x/y)$. Since you can't divide matrices, multiplying by the inverse (which equals the transpose for rotation matrices) works like division.

\begin{eqnarray}
\tilde{\bs{\phi}} &=& R\boxminus R_r \\
\tilde{\bs{\phi}} &=& [\log(R_r^T R)]^\vee
\end{eqnarray}

\begin{eqnarray}
&\exp([\tilde{\bs{\phi}}]^\wedge) = R_r^T R \\
&\exp([\tilde{\bs{\phi}}]^\wedge)R^T = R_r^T \\
\end{eqnarray}

\begin{eqnarray}
R_r &=& R \exp([\tilde{\bs{\phi}}]^\wedge)^T \\
&=& R \exp(([\tilde{\bs{\phi}}]^\wedge)^T) \\
&=& R \exp(-[\tilde{\bs{\phi}}]^\wedge)
\end{eqnarray}

Knowing that $\exp(X) = I + X + \frac{X^2}{2!}+...$ and that $[\tilde{\bs{\phi}}]^\wedge$ is an error term (meaning that it should be small, especially when multiplied by itself),

\begin{eqnarray}
R_r \approx R(I-[\tilde{\bs{\phi}}]^\wedge)
\end{eqnarray}

[TOC](#Table-of-Contents)

### Derivation of Error-State Dynamics
This section will derive the equations for error-state dynamics, needed for MPC control. There are 2 different ways that approximations are made throughout these derivations: 1) the substitution for $R_r$ using a 1st order approximation of a matrix exponential - shown above 2) whenever there are 2 tilde variables multiplied together, they are assumed to be 0 because they should be quite small. Another thing to note is that and the end of the position error-state and velocity error-state derivations, a trick was used to isolate $\tilde{\bs{\phi}}$ by playing with the fact that the hat operator is the same as a cross-product operator. This works for SO(3), but might not work for other manifolds.

#### Position Error State
$\newcommand{\b}[1]{\textbf{#1}}$
\begin{eqnarray}
\dot{\tilde{\b{p}}} &=& \dot{\b{p}} - \dot{\b{p}}_r \\ 
&=& R\b{v} - R_r \b{v}_r \\
&\approx& R\b{v} - R (I-[\tilde{\bs{\phi}}]^\wedge) (\b{v}-\tilde{\b{v}}) \\
&=& R\b{v} - (R-R[\tilde{\bs{\phi}}]^\wedge) (\b{v}-\tilde{\b{v}}) \\
&=& R\b{v} - R\b{v} + R\tilde{\b{v}} + R[\tilde{\bs{\phi}}]^\wedge\b{v} -R[\tilde{\bs{\phi}}]^\wedge \tilde{\b{v}} \\
&\approx& R\tilde{\b{v}} + R[\tilde{\bs{\phi}}]^\wedge \b{v} \\
&=& R\tilde{\b{v}} - R[\b{v}]_\times\tilde{\bs{\phi}}
\end{eqnarray}

#### Attitude Error State
\begin{eqnarray}
\dot{\tilde{\bs{\phi}}} &=& \dot{\bs{\phi}} - \dot{\bs{\phi}}_r \\
&=& \bs{\omega} - \bs{\omega}_r \\
&=& \tilde{\bs{\omega}}
\end{eqnarray}

#### Velocity Error State
\begin{eqnarray}
\dot{\tilde{\b{v}}} &=& \dot{\b{v}} - \dot{\b{v}}_r \\
&=& ([\b{v}]_\times\bs{\omega} - \frac{f}{m}\hat{\b{k}} +R^T\b{g}-\frac{\mu}{m} \b{v}) - ([\b{v}_r]_\times\bs{\omega}_r - \frac{f_r}{m}\hat{\b{k}} +R_r^T\b{g} - \frac{\mu}{m} \b{v}_r) \\
&=& ([\b{v}]_\times\bs{\omega} - \frac{f}{m}\hat{\b{k}} +R^T\b{g}-\frac{\mu}{m} \b{v}) - ([\b{v}-\tilde{\b{v}}]_\times(\bs{\omega}-\tilde{\bs{\omega}}) - \frac{f-\tilde{f}}{m}\hat{\b{k}} +(Rexp(-[\tilde{\bs{\phi}}]^\wedge))^T\b{g} -\frac{\mu}{m} (\b{v}-\tilde{\b{v}})) \\
&=& ([\b{v}]_\times\bs{\omega} - \frac{f}{m}\hat{\b{k}} +R^T\b{g}-\frac{\mu}{m} \b{v}) - ([\b{v}]_\times\bs{\omega}-[\b{v}]_\times\tilde{\bs{\omega}} +[\bs{\omega}]_\times\tilde{\b{v}}+[\tilde{\b{v}}]_\times\tilde{\bs{\omega}} - \frac{f-\tilde{f}}{m}\hat{\b{k}} +exp([\tilde{\bs{\phi}}]^\wedge)R^T\b{g} -\frac{\mu}{m} \b{v} +\frac{\mu}{m}\tilde{\b{v}}) \\
&\approx& ([\b{v}]_\times\bs{\omega} - \frac{f}{m}\hat{\b{k}} +R^T\b{g}-\frac{\mu}{m} \b{v}) - ([\b{v}]_\times\bs{\omega}-[\b{v}]_\times\tilde{\bs{\omega}} +[\bs{\omega}]_\times\tilde{\b{v}}+[\tilde{\b{v}}]_\times\tilde{\bs{\omega}} - \frac{f-\tilde{f}}{m}\hat{\b{k}} +(I+[\tilde{\bs{\phi}}]^\wedge)R^T\b{g} -\frac{\mu}{m} \b{v} +\frac{\mu}{m}\tilde{\b{v}}) \\
&=& ([\b{v}]_\times\bs{\omega} - \frac{f}{m}\hat{\b{k}} +R^T\b{g}-\frac{\mu}{m} \b{v}) - ([\b{v}]_\times\bs{\omega}-[\b{v}]_\times\tilde{\bs{\omega}}+[\bs{\omega}]_\times\tilde{\b{v}}+[\tilde{\b{v}}]_\times\tilde{\bs{\omega}} - \frac{f-\tilde{f}}{m}\hat{\b{k}} +R^T\b{g}+[\tilde{\bs{\phi}}]^\wedge R^T\b{g} -\frac{\mu}{m} \b{v} +\frac{\mu}{m}\tilde{\b{v}}) \\
&=& [\b{v}]_\times\tilde{\bs{\omega}}-[\bs{\omega}]_\times\tilde{\b{v}}-[\tilde{\b{v}}]_\times\tilde{\bs{\omega}} - \frac{\tilde{f}}{m}\hat{\b{k}}-[\tilde{\bs{\phi}}]^\wedge R^T\b{g} -\frac{\mu}{m}\tilde{\b{v}}) \\
&\approx& [\b{v}]_\times\tilde{\bs{\omega}}-[\bs{\omega}]_\times\tilde{\b{v}}+[R^T\b{g}]_\times\tilde{\bs{\phi}} - \frac{\tilde{f}}{m}\hat{\b{k}} - \frac{\mu}{m} \tilde{\b{v}}
\end{eqnarray}

#### Angular Velocity Error State
\begin{eqnarray}
\dot{\tilde{\bs{\omega}}} &=& \dot{\bs{\omega}} - \dot{\bs{\omega}}_r \\
&=& (-J^{-1}[\bs{\omega}]_\times J\bs{\omega} + J^{-1}\bs{\tau}) - (-J^{-1}[\bs{\omega}_r]_\times J\bs{\omega}_r + J^{-1}\bs{\tau}_r) \\
&=& (-J^{-1}[\bs{\omega}]_\times J\bs{\omega} + J^{-1}\bs{\tau}) - (-J^{-1}[\bs{\omega}-\tilde{\bs{\omega}}]_\times J(\bs{\omega}-\tilde{\bs{\omega}}) + J^{-1}(\bs{\tau}-\tilde{\bs{\tau}})) \\
&=& J^{-1}\tilde{\bs{\tau}} -J^{-1}[\bs{\omega}]_\times J\bs{\omega} +J^{-1}[\bs{\omega}-\tilde{\bs{\omega}}]_\times J(\bs{\omega}-\tilde{\bs{\omega}}) \\
&=& J^{-1}\tilde{\bs{\tau}} -J^{-1}[\bs{\omega}]_\times J\bs{\omega} +J^{-1}[\bs{\omega}-\tilde{\bs{\omega}}]_\times J\bs{\omega}-J^{-1}[\bs{\omega}-\tilde{\bs{\omega}}]_\times J\tilde{\bs{\omega}} \\
&=& J^{-1}\tilde{\bs{\tau}} -J^{-1}[\bs{\omega}]_\times J\bs{\omega} +J^{-1}[\bs{\omega}]_\times J\bs{\omega}-J^{-1}[\tilde{\bs{\omega}}]_\times J\bs{\omega}  -J^{-1}[\bs{\omega}]_\times J\tilde{\bs{\omega}} +J^{-1}[\tilde{\bs{\omega}}]_\times J\tilde{\bs{\omega}} \\
&=& J^{-1}\tilde{\bs{\tau}} -J^{-1}[\tilde{\bs{\omega}}]_\times J\bs{\omega}  -J^{-1}[\bs{\omega}]_\times J\tilde{\bs{\omega}} +J^{-1}[\tilde{\bs{\omega}}]_\times J\tilde{\bs{\omega}} \\
&=& J^{-1}\tilde{\bs{\tau}} +J^{-1}[J\bs{\omega}]_\times\tilde{\bs{\omega}}  -J^{-1}[\bs{\omega}]_\times J\tilde{\bs{\omega}} +J^{-1}[\tilde{\bs{\omega}}]_\times J\tilde{\bs{\omega}} \\
&\approx& J^{-1}\tilde{\bs{\tau}} +J^{-1}([J\bs{\omega}]_\times-[\bs{\omega}]_\times J)\tilde{\bs{\omega}}
\end{eqnarray}

[TOC](#Table-of-Contents)

### Summary of Error State Dynamics
These are approximations for the dynamics we will use to derive state-space matrices. We will be using a state-space model for the MPC controller. Remember that state-space form is $\dot{\tilde{\b{x}}} \approx A\tilde{\b{x}}+B\tilde{\b{u}}$.

\begin{eqnarray}
\dot{\tilde{\b{p}}} &\approx& R\tilde{\b{v}} - R[\b{v}]_\times\tilde{\bs{\phi}} \\
\dot{\tilde{\bs{\phi}}} &=& \tilde{\bs{\omega}} \\
\dot{\tilde{\b{v}}} &\approx& [\b{v}]_\times\tilde{\bs{\omega}}-[\bs{\omega}]_\times\tilde{\b{v}}+[R^T\b{g}]_\times\tilde{\bs{\phi}} - \frac{\tilde{f}}{m}\hat{\b{k}} - \frac{\mu}{m} \tilde{\b{v}} \\
\dot{\tilde{\bs{\omega}}} &\approx& J^{-1}\tilde{\b{$\tau$}} +J^{-1}([J\bs{\omega}]_\times-[\bs{\omega}]_\times J)\tilde{\bs{\omega}}
\end{eqnarray}

[TOC](#Table-of-Contents)

### Jacobian Linearization
In order to find the A and B matrices, we will use Jacobian linearization on the equations for error state dynamics. We will partially linearize about equilibrium as will be explained. We will assume that the equilibrium for the attitude of the quadcopter is its current attitude: $R_e = R$. We will not linearize about the reference command $R_r$. This assumption is made due to previous experimentation. I found that the results are the best when $R_e = R$. My thought is that, since I am not really following a trajectory but am using step commands, $R_e$ and $R_r$ both have the z-axis pointing down in an NED coordinate frame. This means that the model used for MPC always thinks the sum of all 4 motor forces acts perfectly upward even though when the vehicle turns, the force vector also rotates. This makes gravity seem stronger due to a loss of force applied in the upward direction and there is a negative coupling effect on the vehicle's height (it will drop or do funny things when the vehicle is tilted). To compensate for this, $R$ is used as $R_e$ since gravity will be properly rotated. All of the other equilibrium values will remain at true equilibrium: $\bs{\tau}_e=\b{v}_e=\bs{\omega}_e=\b{p}_e=[0,0,0]^T, \ \ f_e = mg$.

$$\tilde{\b{x}}=[\tilde{\b{p}},\tilde{\bs{\phi}},\tilde{\b{v}},\tilde{\bs{\omega}}]^T,\ \ \ \ \tilde{\b{u}}=[\tilde{f},\tilde{\bs{\tau}}]^T$$

$\ $

\begin{eqnarray}
A &=& \frac{\partial\dot{\tilde{\b{x}}}}{\partial\tilde{\b{x}}}|_e \\
&\ & \\
&=& \begin{pmatrix} \b{0} & -R_e[\b{v}_e]_\times & R_e & \b{0} \\ \b{0} & \b{0} & \b{0} & I \\ \b{0} & [R_e^T\b{g}]_\times & -\frac{\mu}{m} I-[\bs{\omega}_e]_\times & [\b{v}_e]_\times \\ \b{0} & \b{0} & \b{0} & [J\bs{\omega}_e]_\times-[\bs{\omega}_e]_\times J \end{pmatrix} \\
&\ & \\
&=& \begin{pmatrix} \b{0} & \b{0} & R & \b{0} \\ \b{0} & \b{0} & \b{0} & I \\ \b{0} & [R^T\b{g}]_\times & -\frac{\mu}{m} I & \b{0} \\ \b{0} & \b{0} & \b{0} & \b{0} \end{pmatrix}
&\ & \\
&\ & \\
&\ & \\
B &=& \frac{\partial\dot{\tilde{\b{x}}}}{\partial\tilde{\b{u}}}|_e \\
&\ & \\
&=&\begin{pmatrix} \b{0}_{8x1} & \b{0}_{8x3} \\ -\frac{1}{m} & \b{0}_{1x3} \\ \b{0}_{3x1} & J^{-1} \end{pmatrix}
\end{eqnarray}

If you want to use motor signals as inputs $(\tilde{\b{u}}=[\delta_f,\delta_r,\delta_b,\delta_l]^T)$ with the assumption that $F_*=k_1\delta_*$ and $\tau_*=k_2\delta_*$, then

$$B=\begin{pmatrix} \b{0}_{8x1} & \b{0}_{8x3} \\ -\frac{1}{m} & \b{0}_{1x3} \\ \b{0}_{3x1} & J^{-1} \end{pmatrix} \begin{pmatrix} k_1 & k_1&k_1&k_1 \\ {0} & -\frac{d k_1}{J_x}&{0}&\frac{d k_1}{J_x} \\ \frac{d k_1}{J_y} & {0} & -\frac{d k_1}{J_y}&{0} \\ -\frac{k_2}{J_z} & \frac{k_2}{J_z}&-\frac{k_2}{J_z}&\frac{k_2}{J_z} \end{pmatrix}$$

As a side note, if you want to use h (height as a positive number) as a state rather than down from NED (a negative number) then you just multiply the 3rd row of A by -1. This is how the simulations below were implemented.

[TOC](#Table-of-Contents)

### MPC Design
MPC, which is similar to LQR, was explained in [Unit 1 - Chapter 5: Optimal Control](../Unit-1-SO2/1.5-Optimal-Control.ipynb#Linear-Model-Predictive-Control). As a refresher, MPC uses a convex optimization solver (CVXGEN) with a model (state-space) to predict a defined number of time-steps into the future by varying the input, u. The first step is taken (apply the first set of calculated inputs) and then the process is repeated, predicting the same number of steps into the future starting from the new updated states. The optimization solver minimizes a cost function subject to the provided constraints. 

The time horizon and the discretization time could be chosen in a different way, but I used a time horizon of 10 steps with the size of the step (discretization time) being the settling time of the system divided by the time horizon. This means my MPC controller will take 10 equally spaced steps to predict 4 seconds into the future. Theoretically, better performance would come by increasing the time horizon and decreasing the discretization time; however, this becomes much more computationally expensive. Through some experimentation, I found that a time horizon greater than 10 usually doesn't make much of a difference. This could be due to the fact that MPC only takes the 1st of the predicted steps and then recalculates everything all over again.

\begin{eqnarray}
t_{settle} &=& 5s \\
N &=& 10
\end{eqnarray}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ T_s = \frac{t_{settle}}{N} = 0.5s \ \ \ \  $ where $T_s$ is the discretization time (step size) for the A and B matrices

#### Cost Function

$$J = \sum_{k=0}^N \{(\b{x}[k]-\b{x}_{r})^T W_x (\b{x}[k]-\b{x}_{r})+(\b{u}[k]-\b{u}_{r})^T W_u (\b{u}[k]-\b{u}_{r})\}$$

#### Constraints

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \b{x}[k+1] == A(\b{x}[k]-\b{x}_e) + B(\b{u}[k]-\b{u}_e)\ \ \ $ this says that the system's dynamics must be true

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ u_{min} <= \b{u}[k] <= u_{max}\ \ \ $ this allows MPC to know the input saturation limits

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \b{x}[0]=\b{x}_0\ \ \ $ this allows MPC to know where to start from (current states)

One thing to note is that A needs to be updated at the beginning of each MPC control calculation. Although A is recalculated at every time step, it is a static value in MPC's eyes when predicting into the future. This could potentially be changed where MPC updates A for each step into the future, but that would be more computationally expensive.

[TOC](#Table-of-Contents)

## Simulation

### Framework
Since I had an existing framework built with Simulink, I decided to keep the code in MATLAB rather than redo everything in Python for this notebook. A repo containing the simulations can be found [here](https://github.com/mhask94/Quadcopter_MPC). This repo contains 2 directories: Lie_Group and Euler_Angles. Both directories have a simulation of a quadcopter. The difference is that the plant and controller both model attitude with either Euler angles or the SO(3) Lie Group. Further instruction on how to use the code can be found in the README from the link above.

A diagram of the Simulink framework is shown below. There are 4 signal generators creating square waves as the reference commands for N, E, H, and psi. The reference commands along with the current states are passed into the controller. The controller uses the vectorized error-state for attitude (see [implementation](https://github.com/mhask94/Quadcopter_MPC/blob/master/Lie_Group/control.m#L7)). The controller outputs the 4 throttle commands, 1 for each motor. The control effort is plotted with the upper and lower bounds and is also the input to the dynamics block. The output of the dynamics block is all of the states. Note that there are 18 outputs when there are only 12 states needed to define the system. This is because the dynamics is using all 9 parameters in the rotation matrix rather than the 3 that are needed. This is so that the SO(3) Lie Group kinematics could be modeled: $\dot{R}=R[\bs{\omega}]^\wedge$. The rotation matrix is reshaped into a column vector with 9 elements so that it can be outputted from the s-function modeling the plant (see [implementation](https://github.com/mhask94/Quadcopter_MPC/blob/master/Lie_Group/dynamics.m#L154)). This rotation matrix along with all the other states are fed back into the controller. The current states for position and attitude (N,E,H,R) are also passed into the Draw Quadcopter block which creates the animation as well as plots all of the reference commands with the current respective states.

![Block Diagram of Simulink File](files/Quadcopter_sim_blockDiagram.png)

[TOC](#Table-of-Contents)

### Results

#### Hourglass Waypoint Path Videos
This section has a video for both the SO(3) controller and Euler angle controller showing the responses when commanded to fly an hourglass waypoint path. Besides just North and East, height and psi also have reference commands that change. Both controllers perform very similarly when flying this path. The SO(3) controller has some coupling effects that show up on the psi graph that Euler angles did not have. It is possible that some extra tuning could mitigate that coupling effect on psi, but I didn't want to try and tune gains forever.

Here are the reference commands entered into Simulink for the square wave inputs:

$N_d$: -2.5 m with frequency of .02 Hz and no offset

$E_d$: 2.5 m with frequency of .04 Hz and no offset

$H_d$: 0.5 m with frequency of .01 Hz and a 4.5 m offset

$\psi_d$: $\frac{\pi}{4}$ rad with frequency of .03 Hz and no offset

##### Euler Angles

In [2]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="853" height="480" src="https://www.youtube.com/embed/beyUARSyVbk" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

##### SO(3)

In [1]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="853" height="480" src="https://www.youtube.com/embed/4kyZHC4dqkA" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

[TOC](#Table-of-Contents)

#### Hourglass Waypoint Path Images
This section contains images of the same results from the 2 videos above. They can be used to compare results without having to watch the videos all the way through and remember what the graphs looked like.

##### Euler Angles

![Euler Angles Waypoint Response](files/EA_A-g_Response.png)

##### SO(3)

![Lie Group Waypoint Response](files/LieGroup_A-g_Response.png)

[TOC](#Table-of-Contents)

#### Angle Wrapping Videos
This section has a video for both the SO(3) controller and Euler angle controller showing the responses when commanded to fly to the desired altitude and then test angle wrapping. The commanded value for psi is a full 360+45 degrees. As expected from previous Lie Group discussions, the Euler angle controller will spin all the way around plus the extra 45 degrees while the SO(3) controller only spins the the 45 degrees needed to reach the desired heading.

Here are the reference commands entered into Simulink for the square wave inputs:

$N_d$: 0 m with frequency of .02 Hz and no offset

$E_d$: 0 m with frequency of .04 Hz and no offset

$H_d$: 0.5 m with frequency of .01 Hz and a 4.5 m offset

$\psi_d$: $\frac{9\pi}{4}$ rad with frequency of .03 Hz and no offset

##### Euler Angles

In [66]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="853" height="480" src="https://www.youtube.com/embed/bszRjijxvT4" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

##### SO(3)

In [65]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="853" height="480" src="https://www.youtube.com/embed/dbhbHHqWNvw" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

[TOC](#Table-of-Contents)

#### Angle Wrapping Images

##### Euler Angles

![Euler Angles Wrapping](files/EA_wrapping.png)

##### SO(3)

![Lie Group Wrapping](files/LieGroup_wrapping.png)

[TOC](#Table-of-Contents)

#### Smart Angle Videos
This section shows one of the main advantages of using Lie Groups versus other attitude representations. Lie Groups always travel the shortes distance to get to the desired attitude. To show this, the following videos will use a reference command for psi of 180+30 degrees in a square wave. The Euler angle controller will rotate the 180 degrees plus an extra 30, while the SO(3) controller will initially rotate in the oposite direction 150 degrees to get to the same place. After that, the SO(3) controller will just rotate 60 degrees to the new position when the square wave signal changes rather than spin all the way around like the Euler angle controller.

Here are the reference commands entered into Simulink for the square wave inputs:

$N_d$: 0 m with frequency of .02 Hz and no offset

$E_d$: 0 m with frequency of .04 Hz and no offset

$H_d$: 0.5 m with frequency of .01 Hz and a 4.5 m offset

$\psi_d$: $\frac{7\pi}{6}$ rad with frequency of .03 Hz and no offset

##### Euler Angles

In [71]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="853" height="480" src="https://www.youtube.com/embed/JZvuk1xPpD8" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

##### SO(3)

In [73]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="853" height="480" src="https://www.youtube.com/embed/5Nbp1HTnkKs" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

[TOC](#Table-of-Contents)

#### Smart Angle Images

##### Euler Angles

![Euler Angles Smart Angle](files/EA_smart_angle.png)

##### SO(3)

![Lie Group Smart Angle](files/LieGroup_smart_angle.png)

[TOC](#Table-of-Contents)